In [1]:
import torch
import nltk
import numpy as np
import pandas as pd
import gzip
import torch
import requests
import json
from nltk.corpus import brown
from torch.utils.data import Dataset, DataLoader
import random
import io
from io import open
import unicodedata
import string
import re
import random
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from collections import Counter
import requests

download and prepare the Brown corpus from NLTK for training a Word2Vec model, which is used for creating embeddings of words.

Download Squad2 dataset, split it into training and testing sets, and save them to files.

In [2]:
# Download SQuAD2 dataset
url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json"
response = requests.get(url)

#Save the dataset to a file
dataset_file = "squad2.json"
with open(dataset_file, 'wb') as f:
   f.write(response.content)

#Split dataset into train and test files
train_ratio = 0.8
train_file = "train.json"
test_file = "test.json"

with open(dataset_file, 'r') as f:
    squad_data = json.load(f)
    data = squad_data['data']

    # Shuffle the data
    random.shuffle(data)

    # Split into train and test sets
    split_index = int(len(data) * train_ratio)
    train_data = data[:split_index]
    test_data = data[split_index:]

    # Save train data to file
    train_squad_data = {'data': train_data}
    with open(train_file, 'w') as train_f:
        json.dump(train_squad_data, train_f)

    # Save test data to file
    test_squad_data = {'data': test_data}
    with open(test_file, 'w') as test_f:
        json.dump(test_squad_data, test_f)

In [4]:
#We will use this function on each file in the dataset - test and train
#only the filename is required as the previous function returns the specified file to the same location

def extract_squad_data(file_name):
    with open(file_name, 'r', encoding='utf-8') as f:
        file_name = json.load(f)

    paragraphs = []
    for article in file_name['data']:
        for paragraph in article['paragraphs']:
            for qa in paragraph['qas']:
                question = qa['question']
                answers = [answer['text'] for answer in qa['answers']]
                for answer in answers:
                    paragraphs.append((question, answer))

    df = pd.DataFrame(paragraphs, columns=['Question', 'Answer'])
    return df

In [5]:
#convert .json files to dataframes
train_df = extract_squad_data("train.json") 
test_df = extract_squad_data("test.json")


In [6]:


import pandas as pd
import nltk
import string
from nltk.stem import SnowballStemmer

stemmer = SnowballStemmer("english")

def clean_text(df):
    cleaned_df = df.copy()
    for column in cleaned_df:
        cleaned_df[column] = cleaned_df[column].apply(lambda sentence: process_sentence(sentence))
    return cleaned_df

def process_sentence(sentence):
    sentence = ''.join([s.lower() for s in sentence if s not in string.punctuation])
    sentence = ' '.join(stemmer.stem(w) for w in sentence.split())
    tokens = nltk.tokenize.RegexpTokenizer(r'\w+').tokenize(sentence)

    return ' '.join(tokens) 
# usage:
# df = clean_text(df)


In [7]:
train_clean_df = clean_text(train_df.head(50))


In [ ]:
print(train_clean_df)

In [20]:
class Vocab:
    def __init__(self):
        self.word2index = {}
        self.index2word = {0: "<PAD>", 1: "<SOS>", 2: "<EOS>", 4: "<UNK>"}
        self.word2count = {}
        self.n_words = 4  # Count <PAD>, <SOS>, <EOS>, <UNK>
    def build_vocab(self, sentences):
        for sentence in sentences:
            for word in self.tokenize(sentence):
                self.add_word(word)
    def tokenize(self, sentence):
        return sentence.strip().split()
    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.index2word[self.n_words] = word
            self.word2count[word] = 1
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [21]:
question_vocab = Vocab()
answer_vocab = Vocab()

TypeError: object of type 'Vocab' has no len()

In [22]:
question_vocab = Vocab()
answer_vocab = Vocab()



questions = train_clean_df['Question'].tolist()
answers = train_clean_df['Answer'].tolist()



TypeError: object of type 'Vocab' has no len()

In [11]:
print(question_vocab.index2word)

{0: '<PAD>', 1: '<SOS>', 2: '<EOS>', 4: 'who', 5: 'is', 6: 'the', 7: 'world', 8: 'oldest', 9: 'reign', 10: 'monarch', 11: 'than', 12: 'which', 13: 'queen', 14: 'has', 15: 'elizabeth', 16: 'rule', 17: 'longer', 18: 'how', 19: 'victoria', 20: 'relat', 21: 'to', 22: 'in', 23: 'what', 24: 'year', 25: 'did', 26: 'pass', 27: 'length', 28: 'of', 29: 'histori', 30: 'longest', 31: 'england', 32: 'propos', 33: 'affili', 34: 'with', 35: 'wale', 36: 'quell', 37: 'welsh', 38: 'nation', 39: 'whi', 40: 'britain', 41: 'not', 42: 'want', 43: 'associ', 44: 'conscienti', 45: 'objector', 46: 'join', 47: '1946', 48: 'was', 49: 'alway', 50: 'princ', 51: 'name', 52: 'princess', 53: 'when', 54: 'were', 55: 'and', 56: 'philip', 57: 'marri', 58: 'at', 59: 'famous', 60: 'cathedr', 61: 'mani', 62: 'wed', 63: 'gift', 64: 'receiv', 65: 'design', 66: 'gown', 67: 'former', 68: 'king', 69: 'invit', 70: 'margaret', 71: 'ask', 72: 'do', 73: 'instead', 74: 'mar', 75: 'townsend', 76: 'act', 77: 'would', 78: 'church', 79: 

In [12]:
def pad_or_cut(df, column_names, length):
    pad_symbol = "<PAD>"
    
    for column_name in column_names:
        df[column_name] = df[column_name].apply(lambda x: (x.split()[:length] + [pad_symbol]*length)[:length])
        df[column_name] = df[column_name].apply(lambda x: ' '.join(x))
    return df


TypeError: object of type 'Vocab' has no len()

In [24]:

padded_df = pad_or_cut(train_clean_df, ['Question', 'Answer'], 10)

In [25]:
question_vocab['<UNK>'] = len(question_vocab)

TypeError: object of type 'Vocab' has no len()

In [14]:
padded_df.head()

,Question,Answer
0,who is the world oldest reign monarch <PAD> <P...,elizabeth <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> ...
1,than which queen has elizabeth rule longer <PA...,queen victoria <PAD> <PAD> <PAD> <PAD> <PAD> <...
2,how is victoria relat to elizabeth <PAD> <PAD>...,greatgreatgrandmoth <PAD> <PAD> <PAD> <PAD> <P...
3,in what year did elizabeth pass victoria lengt...,2015 <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>...
4,in the histori of what is elizabeth the longes...,world histori <PAD> <PAD> <PAD> <PAD> <PAD> <P...


In [15]:
question_vocab['<UNK>'] = len(question_vocab)
answer_vocab['<UNK>'] = len(answer_vocab)

TypeError: object of type 'Vocab' has no len()

In [16]:
def words_to_indices(df, question_vocab, answer_vocab):
    df['Question'] = df['Question'].apply(lambda sentence: [question_vocab.word2index[word] if word in question_vocab.word2index else question_vocab.word2index['<UNK>'] for word in sentence.split()])
    df['Answer'] = df['Answer'].apply(lambda sentence: [answer_vocab.word2index[word] if word in answer_vocab.word2index else answer_vocab.word2index['<UNK>'] for word in sentence.split()])
    return df

In [17]:
indices_df = words_to_indices(padded_df, question_vocab, answer_vocab)

KeyError: '<UNK>'

In [75]:
indices_df.head()

,Question,Answer
0,"[4, 5, 6, 7, 8, 9, 10, 7, 11, 12]","[4, 5, 6, 6, 6, 6, 6, 6, 6, 6]"
1,"[6, 4, 5, 13, 7, 8, 14, 15, 16, 16]","[4, 5, 6, 6, 6, 6, 6, 6, 6, 6]"
2,"[4, 5, 10, 17, 18, 7, 19, 20, 21, 7]","[4, 5, 6, 6, 6, 6, 6, 6, 6, 6]"
3,"[4, 22, 5, 23, 7, 24, 25, 21, 26, 27]","[4, 5, 6, 6, 6, 6, 6, 6, 6, 6]"
4,"[7, 28, 29, 30, 27, 10, 17, 31, 32, 5]","[4, 5, 7, 6, 6, 6, 6, 6, 6, 6]"
